In [ ]:
# -*-   Coding with utf-8   -*- #
# -*- Developed by Harryjin -*- #

In [1]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models
from torch.autograd import Variable
import pandas as pd
from net import *
from tqdm import tqdm

C:\Users\luke0\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Basic Traning Settings
BATCH_SIZE = 64
EPOCHS = 30
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda


In [3]:
# Data transformation
transform = transforms.Compose([      
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])

])
transform_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [4]:
# Read Data
dataset_train = datasets.ImageFolder('./dataset/classification/train', transform)
print(dataset_train.imgs)
# Read the label
print(dataset_train.class_to_idx)
dataset_test = datasets.ImageFolder('./dataset/classification/val', transform_test)
# Read the label
print(dataset_test.class_to_idx)

[('./dataset/1/train\\no\\004505-20200415@095140-R1-S.jpg', 0), ('./dataset/1/train\\no\\004508-20200415@101356-L2-S.jpg', 0), ('./dataset/1/train\\no\\004508-20200415@101356-R1-S.jpg', 0), ('./dataset/1/train\\no\\004511-20200415@113206-L2-S.jpg', 0), ('./dataset/1/train\\no\\004511-20200415@113206-R1-S.jpg', 0), ('./dataset/1/train\\no\\004516-20200415@135154-L2-S.jpg', 0), ('./dataset/1/train\\no\\004516-20200415@135154-R1-S.jpg', 0), ('./dataset/1/train\\no\\004518-20200415@143356-R1-S.jpg', 0), ('./dataset/1/train\\no\\004519-20200415@150647-R1-S.jpg', 0), ('./dataset/1/train\\no\\004527-20200416@111117-R1-S.jpg', 0), ('./dataset/1/train\\no\\004530-20200416@124129-L2-S.jpg', 0), ('./dataset/1/train\\no\\004530-20200416@124129-R1-S.jpg', 0), ('./dataset/1/train\\no\\004542-20200417@110630-L2-S.jpg', 0), ('./dataset/1/train\\no\\004548-20200417@145257-L2-S.jpg', 0), ('./dataset/1/train\\no\\004553-20200417@160716-L2-S.jpg', 0), ('./dataset/1/train\\no\\004561-20200418@092531-L3-S.j

In [5]:
# Load the data using DataLoader
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers = 1)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, num_workers = 1)
modellr = 1e-4

In [ ]:
# Resnet18

# Instantiate the model
criterion = nn.CrossEntropyLoss() # Loss function

model = torchvision.models.resnet18(pretrained=True) # Use ResNet18
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.to(DEVICE)
# Using the most powerful optimizer - Adam
optimizer = optim.Adam(model.parameters(), lr=modellr)

In [ ]:
# Resnet50

# Instantiate the model
criterion = nn.CrossEntropyLoss()

model = torchvision.models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.to(DEVICE)
# Optimizer Adam
optimizer = optim.Adam(model.parameters(), lr=modellr)

In [ ]:
# Resnet101

# Instantiate the model
criterion = nn.CrossEntropyLoss()

model = torchvision.models.resnet101(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.to(DEVICE)
# Optimizer Adam
optimizer = optim.Adam(model.parameters(), lr=modellr)

In [ ]:
# CustomCNNReLU Finished

# Instantiate the model
criterion = nn.CrossEntropyLoss()

model = CustomCNNReLU() # Inside net

model.to(DEVICE)
# Optimizer Adam
optimizer = optim.Adam(model.parameters(), lr=modellr)

'''
Result of ReLU
Learning rate: 1e-3
Total Epochs: 30
Accuracy: 85% at most

Summary: 
ReLU is one of the best activation functions in deep learning with delighted results using low learning rate and small epochs amount
'''

In [7]:
# CustomCNNSigmoid

# Instantiate the model
criterion = nn.CrossEntropyLoss()

model = CustomCNNSigmoid()

model.to(DEVICE)
# Optimizer Adam
optimizer = optim.Adam(model.parameters(), lr=modellr)

EPOCHS = 100
modelllr = 0.1
# Due the speciality of sigmoid (No very easy to spread backward)
# The total epochs should be way higher and the learning rate should be higher than the ReLU

'''
Result of Sigmoid (Test 01)
Learning rate: 1e-3
Total Epochs: 30
Accuracy: 36%

Summary:
Sigmoid is one of the earliest used activation functions in deep learning as well as one of the worst one.
Sigmoid can not separate the data effectively causing the bad result
'''

'\nResult of Sigmoid (Test 01)\nLearning rate: 1e-3\nTotal Epochs: 30\nAccuracy: 36%\n\nSummary:\nSigmoid is one of the earliest used activation functions in deep learning as well as one of the worst one.\nSigmoid can not separate the data effectively causing the bad result\n'

In [7]:
df_train = pd.DataFrame(columns = ['epoch', 'train_loss', 'val_loss', 'accuracy'])

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 20 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 20))
    # print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew


# Train method

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_loss = 0
    total_num = len(train_loader.dataset)

    loop = tqdm(enumerate(train_loader), total = len(train_loader)) # Showing progress bar with tqdm
    for batch_idx, (data, target) in loop:
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)

        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
        loop.set_description(f'Epoch [{epoch}/{EPOCHS}]')
        loop.set_postfix_str('loss = {:6f}'.format(loss.item()))
    ave_loss = sum_loss / len(train_loader)
    
    torch.save(model, './checkpoints/model_{}.pth'.format(epoch)) # Save every epoch
    return ave_loss 

# Validate the model every epoch
def val(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data)
            loss = criterion(output, target)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_loader)
        print('Val set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))
        return acc, avgloss

In [8]:
# Train
for epoch in range(1, EPOCHS + 1):
    adjust_learning_rate(optimizer, epoch)
    loss = train(model, DEVICE, train_loader, optimizer, epoch)
    acc, aveloss = val(model, DEVICE, test_loader)
    df_train.loc[len(df_train.index)] = [epoch, loss, aveloss, acc] # Save the traning and validation data every epoch to a csv
torch.save(model, 'model.pth') # Save the model
df_train.to_csv('loss.csv')

Epoch [1/30]: 100%|██████████| 12/12 [00:48<00:00,  4.05s/it, loss = 0.299207]


Val set: Average loss: 0.6217, Accuracy: 211/333 (63%)



Epoch [2/30]: 100%|██████████| 12/12 [00:26<00:00,  2.21s/it, loss = 0.149505]


Val set: Average loss: 0.3930, Accuracy: 268/333 (80%)



Epoch [3/30]: 100%|██████████| 12/12 [00:26<00:00,  2.18s/it, loss = 0.024050]


...

Val set: Average loss: 0.3793, Accuracy: 287/333 (86%)



Epoch [29/30]: 100%|██████████| 12/12 [00:35<00:00,  2.98s/it, loss = 0.000469]


Val set: Average loss: 0.3820, Accuracy: 288/333 (86%)



Epoch [30/30]: 100%|██████████| 12/12 [00:38<00:00,  3.17s/it, loss = 0.000518]


Val set: Average loss: 0.3763, Accuracy: 288/333 (86%)



In [ ]:
import pandas as pd
%matplotlib inline

# Show the traning result with matplotlib
df = pd.read_csv('./loss.csv')

df[['train_loss', 'val_loss', 'accuracy']].plot()